In [8]:
import pandas as pd
import numpy as np
import warnings
import re

warnings.simplefilter('ignore')

#read MDALL dataset 
data_archive=pd.read_json('archive_device_identifier.json')
data_active=pd.read_json('active_device_identifier.json')

data_archive_sel=data_archive[['original_licence_no','device_id','device_identifier']]
data_active_sel=data_active[['original_licence_no','device_id','device_identifier']]
#combine above two data sets into one
d_mdall=data_archive_sel.append(data_active_sel)

#read guidid dataset
data_device=pd.read_csv('AccessGUDID/device.txt',sep='|', error_bad_lines=False, index_col=False, dtype='unicode')
# let us minimize the columns to analysis
data_device_sel=data_device[['brandName','versionModelNumber','catalogNumber','companyName','deviceDescription']]


In [9]:
#mdall dataset preparation

data_active_license=pd.read_json('active_license.json')
data_archive_license=pd.read_json('archive_license.json')

#let us append active and arhcive

d_license=data_active_license.append(data_archive_license)
d_license_sel=d_license[['original_licence_no','licence_name','company_id']]
#select interesting columns only
d_license_sel=d_license[['original_licence_no','licence_name','company_id']]
#same logic for company
data_company=pd.read_json('company.json')
d_company_sel=data_company[['company_id','company_name']]

#now merge on key companyid

license_all_with_company=pd.merge(d_license_sel,d_company_sel,on="company_id")
dmall_complete=pd.merge(d_mdall,license_all_with_company,on='original_licence_no')

In [10]:
#clean the special charcters from both data sets on catlognumber
#data quality


dmall_complete['device_identifier']=dmall_complete['device_identifier'].map(lambda x: re.sub(r'^[\t]+|[ \t]+$', '', x))
data_device_sel['catalogNumber']=data_device_sel['catalogNumber'].astype(str)
data_device_sel['catalogNumber']=data_device_sel['catalogNumber'].map(lambda x: re.sub(r'^[\t]+|[ \t]+$', '', x))
#data_device_sel['versionModelNumber']=data_device_sel['versionModelNumber'].astype(str)
#data_device_sel['versionModelNumber']=data_device_sel['versionModelNumber'].map(lambda x: re.sub(r'\W+', '', x))


In [11]:
#change the column heading of company_name to d_company_name

dmall_complete.rename(columns = {'company_name':'dmall_company_name'}, inplace = True) 
#create master list
overall_df=pd.merge(dmall_complete,data_device_sel,left_on='device_identifier',right_on='catalogNumber')




In [12]:
#Data quality
#let us start rempoving duplicate

duplicate_in_overall_df = overall_df.duplicated(subset=['catalogNumber'])

## keeping the category row for which company name matches to break the duplicate 
## catalognumber
## rest is kept the first one matched

prev=False

# add Temp column to keep tracof of what needs to be deleted
#please note this takes a bit long to execute
overall_df['Temp']=False
count=len(overall_df.index)
print(count)
i=0
while i< count:
    
    
    prev=duplicate_in_overall_df.iloc[i]
    if (prev==False):
        i+=1
    while prev:
        if (overall_df.iloc[i]['companyName']==overall_df.iloc[i]['dmall_company_name']):
                #print(i,overall_df.iloc[i]['companyName'],overall_df.iloc[i]['dmall_company_name'])
                i+=1
                prev=duplicate_in_overall_df.iloc[i]
                while prev:
                    overall_df.at[i,'Temp']=True
                    i+=1
                    prev=duplicate_in_overall_df.iloc[i]
                    
                
        else:
            overall_df.at[i,'Temp']=True
            #print(overall_df.iloc[i]['Temp'])
            
        i+=1
        if (i >=count):
            break
        prev=duplicate_in_overall_df.iloc[i]
            
        
  
    


            
        

overall_df.drop(overall_df.loc[overall_df['Temp']==True].index,inplace=True)
#second round duplicate teatment, this time we keep the last one

overall_df.drop_duplicates(subset="catalogNumber",keep="last",inplace=True)


 

647661


In [13]:
#Output 1
#instruction run the program and enter catlog number
#todo if not found
#1013470-180 -test case

name=input("search for a catalog Number ")
print("Catalog number    :", name)
print("MDALL Description  :",
      overall_df[overall_df['device_identifier']==name].licence_name.to_string(index=False))
print("GUIDID Description :",
      overall_df[overall_df['device_identifier']==name].deviceDescription.to_string(index=False))

search for a catalog Number 1013470-180
Catalog number    : 1013470-180
MDALL Description  :  ARMADA 18 PTA CATHETER
GUIDID Description :  Armada 18 PTA Catheter 6.0 mm x 180 mm x 150 c...


In [14]:
#output 2


data_device_sel

catboth=np.count_nonzero(overall_df['device_identifier'].unique())
total_dmall_catalog=np.count_nonzero(dmall_complete['device_identifier'].unique())
total_guidid_catalog=np.count_nonzero(data_device_sel['catalogNumber'].unique())

print("guid total unque catalog  :",total_guidid_catalog)
print("mdall total unque catalog :",total_dmall_catalog)
print("---------------output2 as in assignment-------")
print("guidid-only catalog count:",total_guidid_catalog-catboth)
print("mdall-only catalog count:",total_dmall_catalog-catboth)
print("catalogs in both datasets: ",catboth)


#df_list=df.values.tolist()
#df_list

guid total unque catalog  : 1309056
mdall total unque catalog : 1886635
---------------output2 as in assignment-------
guidid-only catalog count: 984169
mdall-only catalog count: 1561748
catalogs in both datasets:  324887
